In [ ]:
import uuid
import random
from datetime import datetime, timedelta
import math
import time
import requests

acceleration = 7.72 # max acceleration 0 to 100 km/h in 3.6 seconds
deceleration = 5  # Normally it is 2.5 m/s^2 but for simulating purposes since only its gonna speed up we can increase the deceleration some portion.
# Deceleration is the avg backward acceleration

start_speed = 0 # Start speed 0 km/h
speed_update_seconds = 1 # its not getting used but we designed with the idea of it

# Istanbul Coordinates
start_location = {"latitude" : 41.0082, "longitude" : 28.9784}
start_time = datetime.now()

device_id = uuid.uuid4()

speed = 0

OpenWeather_API_KEY = "get_api_key" # Check the planning. You can use 1000 requests per day freely.

In [2]:
def get_speed_change():
    is_speeding = random.choice([True, False])

    if is_speeding:
        return random.uniform(0, acceleration)
    else:
        return -random.uniform(0, deceleration)

In [3]:
def get_direction(): ## Lets assume that the rotation and angle values are coming from sensors that updates the absolute location.

    random_x = random.choice(['E', 'W', 'None'])  # Randomly choose East or West
    random_y = random.choice(['N', 'S', 'None'])  # Randomly choose North or South
    angle = random.uniform(0, 360)
    return [random_x, random_y, angle]

In [4]:
def simulate_vehicle_movement(speed):
    global start_location
    
    direction = get_direction()
    
    speed_m_s = speed / 3.6 # Convert km/h to m/sn

    direction_x, direction_y = direction[0], direction[1]

    angle_radians = math.radians(direction[2])

    horizontal = speed_m_s * math.cos(angle_radians)
    vertical = speed_m_s * math.sin(angle_radians)
    
    # Calculate the change in latitude and longitude
    if direction_x == 'E':
        start_location['longitude'] += horizontal / (111320 * math.cos(math.radians(start_location['latitude'])))
    elif direction_x == 'W':
        start_location['longitude'] -= horizontal / (111320 * math.cos(math.radians(start_location['latitude'])))
    
    if direction_y == 'N':
        start_location['latitude'] += vertical / (111320)
    elif direction_y == 'S':
        start_location['latitude'] -= vertical / (111320)
    
    return start_location

In [5]:
def generate_vehicle_data(device_id, timestamp):
    global start_time, start_speed

    speed_change = get_speed_change()
    start_speed = start_speed + speed_change

    if start_speed >= 250:
        start_speed = 250
    
    if start_speed < (-deceleration):
        start_speed = (-deceleration)
    

    
    # Generate and return the vehicle data dictionary
    return {
        'deviceId': str(device_id),
        'timestamp': timestamp,
        'direction': [get_direction()[0], get_direction()[1]],
        'speed': start_speed,
        'fuelType': "Hybrid",
        'brand': 'Audi',
        'model': 'RS7',
        'engineType': 'V8',
    }

In [6]:
def get_gps_data(speed, device_id, timestamp):

    location = simulate_vehicle_movement(speed)

    return {
        'deviceId': str(device_id),
        'timestamp': timestamp,
        'latitude': location['latitude'],
        'longitude': location['longitude']
    }

In [7]:
def get_weather_data(latitude, longitude, timestamp):
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={latitude}&lon={longitude}&appid={OpenWeather_API_KEY}"

    response = requests.get(url)
    data = response.json()

    kelvin_temp = data['main']['temp']

    celcius_temp = kelvin_temp - 273.15

    return {
        'timestamp': timestamp,
        'temperature': celcius_temp,
        'humidity': data['main']['humidity'],
        'wind_speed': data['wind']['speed']
    }

In [8]:
def simulate_vehicle_data(device_id):
    try:
        while True:
            timestamp = (datetime.now() + timedelta(seconds=1)).strftime("%H:%M:%S")

            vehicle_data = generate_vehicle_data(device_id, timestamp)

            gps_data_send = get_gps_data(vehicle_data['speed'], device_id, timestamp)

            weather_data_send = get_weather_data(gps_data_send['latitude'], gps_data_send['longitude'], timestamp)

            print("Vehicle Data:", vehicle_data)
            # print("Speed Data:", vehicle_data['speed'])
            print("GPS Data:", gps_data_send)
            print("Weather Data:", weather_data_send)

            time.sleep(1)
    except KeyboardInterrupt:
        print("\nSimulation stopped by user.")

In [10]:
simulate_vehicle_data(device_id)

Vehicle Data: {'deviceId': 'b3be5270-b679-46d2-b133-af5cd3e72ab0', 'timestamp': '16:52:11', 'direction': ['None', 'None'], 'speed': -3.091470630212814, 'fuelType': 'Hybrid', 'brand': 'Audi', 'model': 'RS7', 'engineType': 'V8'}
GPS Data: {'deviceId': 'b3be5270-b679-46d2-b133-af5cd3e72ab0', 'timestamp': '16:52:11', 'latitude': 41.00819403978185, 'longitude': 28.978393510107416}
Weather Data: {'timestamp': '16:52:11', 'temperature': 19.79000000000002, 'humidity': 56, 'wind_speed': 2.06}
Vehicle Data: {'deviceId': 'b3be5270-b679-46d2-b133-af5cd3e72ab0', 'timestamp': '16:52:13', 'direction': ['None', 'N'], 'speed': -5, 'fuelType': 'Hybrid', 'brand': 'Audi', 'model': 'RS7', 'engineType': 'V8'}
GPS Data: {'deviceId': 'b3be5270-b679-46d2-b133-af5cd3e72ab0', 'timestamp': '16:52:13', 'latitude': 41.00819403978185, 'longitude': 28.97838043047893}
Weather Data: {'timestamp': '16:52:13', 'temperature': 19.79000000000002, 'humidity': 56, 'wind_speed': 2.06}
Vehicle Data: {'deviceId': 'b3be5270-b679-